# Modeling a System and an Intervention

In this module, we'll go deeper with a single problem and we'll try to...
* consider modeling strategies
* identify an existing model as a starting point
* implement the model and discuss calibration
* propose an intervention
* plan experiments to model the costs and benefits of the intervention

## Crop disease

Crop disease is a common agricultural challenge. In the absence of targeted (e.g., pharmaceutical or chemical) solutions, we might take a step back and see if there are some systems characteristics we can identify and use to propose additional options.

### Modeling strategies

There are lots of ways we might model the crop disease phenomenon.

Let's start with some abstractions:
* consider a square plot of land
* subdivided into many smaller, uniform sections
* where each section is sufficiently small that
    * it either only includes one plant, or else is small enough that all of the plants in the cell are likely to either be healthy or diseased
* disease can spread locally but not directly over large distances

This is a *spatial* model with local, short-term interaction patterns. We might model this with a 2-D cellular automaton.

Of course this is oversimplified. But remember
* we know that extremely large, complicated models can produce phenomena of interest
    * but our ability to work with those models -- espcially for causal interventions -- is limited
* the goal here is to come up with a parsimonious generative model that might produce the outcome we're addressing



### Forest fire model

Our crop damage model is similar to an existing well-known model: the Forest Fire Cellular Automaton, which is a simple computational model used to study the dynamics of forest fires.

The Forest Fire model was introduced by Drossel and Schwabl in 1992.
* the model is interesting because it can generate clusters of trees that are power-law distributed
    * meaning that there are many small clusters and a few large ones
    * this feature is observed in many natural and social systems
* the Forest Fire model has been used to study not only the dynamics of forest fires
    * but also other phenomena that exhibit similar patterns, such as the spread of diseases or information through a network

The Forest Fire Cellular Automaton model typically has three states for each cell: empty, tree, and burning. At each time step, the model updates the state of each cell based on the following rules:

1. A burning cell turns into an empty cell.
2. A tree will become a burning cell if at least one neighbor is burning.
3. An empty space becomes a tree with probability `p`.
4. A tree ignites, turning into a burning cell with probability `f`, regardless of its neighboring cells.

In this model, `p` is the tree growth probability and `f` is the lightning strike probability, representing the chance of a tree spontaneously catching fire. These probabilities are typically small.

Let's implement a basic model:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the states
EMPTY, TREE, BURNING = 0, 1, 2

# Define the probabilities
p, f = 0.03, 0.001  # probabilities of tree growth and fire

# Grid size
size = (100, 100)

# Initialize grid: all cells start as empty
grid = np.zeros(size, dtype=int)

The update function implements the CA rules

In [ ]:
def update(grid):
    new_grid = grid.copy()
    for i in range(size[0]):
        for j in range(size[1]):
            if grid[i, j] == EMPTY and np.random.rand() < p:
                new_grid[i, j] = TREE
            elif grid[i, j] == TREE:
                if np.random.rand() < f:
                    new_grid[i, j] = BURNING
                elif any(grid[ii, jj] == BURNING for ii in range(max(i-1, 0), min(i+2, size[0]))
                                               for jj in range(max(j-1, 0), min(j+2, size[1]))):
                    new_grid[i, j] = BURNING
            elif grid[i, j] == BURNING:
                new_grid[i, j] = EMPTY
    return new_grid

To make the output easier to interpret, we can define a custom color mapping where
* black = empty
* gree = tree
* orange = fire


Note that for implementation simplicity, this mapping is only accurate once all 3 states are present in the space, which typically happens after the first 2-3 iterations.

In [ ]:
from matplotlib.colors import ListedColormap

cmap = ListedColormap(['black', 'green', 'orange'])

In [ ]:
from IPython import display
import time

n_steps = 100

for step in range(n_steps):
    grid = update(grid)
    display.clear_output(wait=True)
    plt.matshow(grid, cmap=cmap)
    plt.title(f"Step {step} of {n_steps}")
    plt.show()

If the tree growth (or crop replacement) is much faster, it creates more fuel at the same time, so we get another dynamic

In [ ]:
p, f = 0.35, 0.001  # probabilities of tree growth and fire

grid = np.zeros(size, dtype=int)

In [ ]:
for step in range(n_steps):
    grid = update(grid)
    display.clear_output(wait=True)
    plt.matshow(grid, cmap=cmap)
    plt.title(f"Step {step} of {n_steps}")
    plt.show()

This result looks a lot more like pure noise ... perhaps there is a tipping point probability that represents a boundary between these outcomes. 

That would be an interesting investigation, but today we'll return to the original configuration and look at how we might limit the disease spread in a traditional and practical way.

### Intervention

An imperfect intervention that is often applied to forest fires -- and might be suitable for a crop disease experiment -- is the creation of "fire breaks."

Fire breaks are trenches, roads, or other areas of bare ground cut through the forest in order to make it harder for fire to spread. These fire breaks are imperfect because wind can blow burning material across the break. In our model, though, the CA rules don't allow for failures of breaks, because they only address neighboring cells.

* Could we model stochastic fire break failure? What would that look like?
* What are some advantages and disadvantages to adding that additional realistic element to our model?

For now, we'll keep the model as it is and consider the effects of breaks:

How effective are they? Before we can even answer that, we need to think about how to measure effectiveness.
1. We can look at prior- and post- intervention damage ratios
1. But we also need to consider the cost: each break represents an area with no crops
1. Consider the limit case: breaks everywhere, disease nowhere ... but no crops either. So we need to consider the cost of the intervention.

### Lab Project

1. Choose an initial configuration of the model -- to keep things simple, you may want to fix the step count -- and calculate the healthy yield at the end of the iterations.
2. Choose a fire break design or pattern to experiment with -- it should be something easily scalable, such as a single north-south break in the center (which could be iterated by subdividing the remaining areas and adding identical breaks)
3. For each level of intervention, measure
    1. healthy yield at the end of the sequence
    2. cost of the intervention (measured in cells or land area)
4. Plot the yield against the intervention level

When we're done, we'll discuss the results.